In [2]:
import os
os.chdir("/content/drive/My Drive/Mamba-Proj")

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from mamba import Mamba, MambaConfig
import argparse
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')

ModuleNotFoundError: No module named 'numpy'

In [21]:
parser = argparse.ArgumentParser()
parser.add_argument('--use-cuda', default=False,
                    help='CUDA training.')
parser.add_argument('--seed', type=int, default=1, help='Random seed.')
parser.add_argument('--epochs', type=int, default=20,
                    help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.01,
                    help='Learning rate.')
parser.add_argument('--wd', type=float, default=1e-5,
                    help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=16,
                    help='Dimension of representations')
parser.add_argument('--layer', type=int, default=2,
                    help='Num of layers')
parser.add_argument('--n-test', type=int, default=500,
                    help='Size of test set')
#parser.add_argument('--ts-code', type=str, default='601988',
#                    help='Stock code')
parser.add_argument('-f', type=str, default='/content/drive/My Drive/Mamba-Proj',
                    help='current directory')

_StoreAction(option_strings=['-f'], dest='f', nargs=None, const=None, default='/content/drive/My Drive/Mamba-Proj', type=<class 'str'>, choices=None, required=False, help='current directory', metavar=None)

In [5]:
args = parser.parse_args()
args.cuda = args.use_cuda and torch.cuda.is_available()

In [6]:
def set_seed(seed,cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)

def dateinf(series, n_test):
    lt = len(series)
    print('Training start',series[0])
    print('Training end',series[lt-n_test-1])
    print('Testing start',series[lt-n_test])
    print('Testing end',series[lt-1])

set_seed(args.seed,args.cuda)

In [130]:
class Net(nn.Module):
    def __init__(self,in_dim,out_dim):
        super().__init__()
        self.config = MambaConfig(d_model=args.hidden, n_layers=args.layer)
        self.mamba = nn.Sequential(
            nn.Linear(in_dim,args.hidden),
            Mamba(self.config),
            nn.Linear(args.hidden,out_dim),
            #-----------------------------
            nn.ReLU()
            #-----------------------------
        )

    def forward(self,x):
        x = self.mamba(x)
        return x.flatten()



def BL_loss(data, output):
    #feature data is M, t, x | 'label' is u
    data = data.squeeze(0)  # Now tensor has shape (500, 2)
    #print(data.shape)
    #print(output.shape)
    #print(data[:,5])
    tot_data = torch.column_stack((data, output)).requires_grad_()
    tot_data.retain_grad()
    #print(tot_data.shape)
    sum = 0
    # ----- Initial Condition ----
    init_data = tot_data[np.where(data[:,1]==0)] # observations at time == 0
    print(init_data.shape)
    init_preds = init_data[:,3]
    IC_loss = torch.mean(init_preds**2) # penalizing where init_preds @t=0 != 0
    sum += IC_loss
    IC_loss = None # clear up memory

    # ----- Boundary Condition ----
    boundary_data = tot_data[np.where(data[:,2]==0)] # observations at x == 0
    boundary_preds = boundary_data[:,3]
    BC_loss = torch.mean((boundary_preds - 1)**2) # penalizing where init_preds @x=0 != 1
    sum += BC_loss
    BC_loss = None
    # ----- Residual Loss ----
    for M in np.unique(data[:,0]):  # Ms = {2,4, ..., 100}
        # Automatic differentiation to compute derivatives
        Mdata = tot_data[np.where(data[:,0] == M)].requires_grad_() # data for this M
        print(Mdata.shape)
        outputs = (Mdata[:,3]).requires_grad_() # select outputs corresponding to the specific M
        print(outputs.shape)
        # ------R1 ------
        ts = (Mdata[:,1]).requires_grad_()
        u_t = torch.autograd.grad(outputs, ts, grad_outputs=torch.ones_like(outputs), allow_unused=True)[0] #create_graph=True

        """
        # IF autograd method doesnt work: implemented the approximation
        R1 = torch.zeros((len(output)-1),len(np.unique(Mdata[:,1])))
        ts = np.unique(Mdata[:,1])
        xs = np.unique(Mdata[:,2])
        for i in range(1, len(ts)-1):        # looping thru t's
            for j in range(1, len(xs)-1):    # looping thru x's
                pre_u = Mdata[np.where((Mdata[:, 1] == ts[1-i]) * (Mdata[:, 2] == xs[i]))] #select row that satisfies both conditions
                pre_u = pre_u[3]

                post_u = Mdata[np.where((Mdata[:, 1] == ts[1+i]) * (Mdata[:, 2] == xs[i]))] #select row that satisfies both conditions
                post_u = post_u[3]

                R1[i,j] = (post_u - pre_u)/(ts[1+i] - ts[1-i])

        # ---- R2 -----
        f_theta = []
        for i in range(len(xs)):
            arr =

        f_theta = np.array(f_theta)
        for i in range(1, len(ts)-1):        # looping thru t's
            for j in range(1, len(xs)-1):    # looping thru x's



        """
        # ------R2 ------
        # Define the flux function and its derivative

        f = (outputs**2 / (outputs**2 + (1/M)*(1-outputs)**2)).requires_grad_()
        print(f.shape)
        xs = (Mdata[:,2]).requires_grad_()
        f_x = torch.autograd.grad(f, xs, grad_outputs=torch.ones_like(f), allow_unused=True)[0]
        print(f_x, u_t)

        sum += (np.linalg.norm(u_t + f_x))**2

    return torch.tensor(sum, dtype=torch.float32)

In [129]:
def PredictWithData(trainX, testX):
    clf = Net(len(trainX[0]),1)
    opt = torch.optim.Adam(clf.parameters(),lr=args.lr,weight_decay=args.wd)
    xt = torch.from_numpy(trainX).float().unsqueeze(0)
    xv = torch.from_numpy(testX).float().unsqueeze(0)
    #yt = torch.from_numpy(trainy).float()
    if args.cuda:
        clf = clf.cuda()
        xt = xt.cuda()
        xv = xv.cuda()
        yt = yt.cuda()

    for e in range(args.epochs):
        clf.train()
        z = clf(xt)
        #print(z.shape)
        z = z.clone().detach().requires_grad_()
        z.retain_grad()
        #-----------------------------------------------------------
        loss = BL_loss(xt, z) # NEW LOSS
        #-----------------------------------------------------------
        opt.zero_grad()
        print("Backward is reached")
        loss.backward()
        opt.step()
        if e%10 == 0 and e!=0:
            print('Epoch %d | Lossp: %.4f' % (e, loss.item()))

    clf.eval()
    mat = clf(xv)
    if args.cuda: mat = mat.cpu()
    yhat = mat.detach().numpy().flatten()
    return yhat

In [66]:
train_dat = pd.read_csv('final_train_data.csv')
print(train_dat.head())
test_dat = pd.read_csv('test_data_new.csv')

     M  t      x    u
0  2.0  0  0.000  1.0
1  2.0  0  0.003  0.0
2  2.0  0  0.006  0.0
3  2.0  0  0.009  0.0
4  2.0  0  0.012  0.0


In [69]:
trainY = train_dat.iloc[0:500,3].values
#print(trainY.head())
trainX = train_dat.iloc[0:500,:3].values
#print(trainX.shape)

In [70]:
testY = test_dat.iloc[0:100,3].values
testX = test_dat.iloc[0:100,0:3].values

In [71]:
print(f"Shape of trainX: {trainX.shape}")
print(f"Shape of testX: {testX.shape}")

Shape of trainX: (500, 3)
Shape of testX: (100, 3)


In [13]:
trainX.size

5577800

In [131]:
predictions = PredictWithData(trainX, testX)

torch.Size([500, 4])
torch.Size([500, 4])
torch.Size([500])
torch.Size([500])
None None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'